In [267]:
import os
import sys
import pickle
import re

### Create dataset for MBPP

In [248]:
SELF_EVAL_PROMPT = '''Instruction: Please evaluate the truthfulness of the proposed answer based on the given problem and internal knowledge.

Problem: You are an expert Python programmer, and here is your task: Write a function to find the similar elements from the given two tuple lists. Your code should pass these tests:
assert similar_elements((3, 4, 5, 6),(5, 7, 4, 10)) == (4, 5)
assert similar_elements((1, 2, 3, 4),(5, 4, 3, 7)) == (3, 4)
assert similar_elements((11, 12, 14, 13),(17, 15, 14, 13)) == (13, 14)
Proposed Answer:
def similar_elements(test_tup1, test_tup2):
  res = tuple(set(test_tup1) & set(test_tup2))
  return (res)

Is the proposed answer:
A. True
B. False
The proposed answer is: A. True

Problem: [PROBLEM]
Proposed Answer: [ANSWER]
Is the proposed answer:
A. True
B. False
The proposed answer is:'''

In [249]:
problems = pickle.load(open('/hcode/mbpp_problems.pickle', 'rb'))

In [250]:
def remove_few_shot(p):
    c = problems['MBPP/0']['prompt'].find('[DONE]')
    return p[c:].replace('[BEGIN]', '').replace('[DONE]', '').strip('\n')

In [252]:
len(general_prompt)

1761

In [253]:
NUM_TASKS = 510

In [254]:
tasks = []

for i in range(NUM_TASKS):
    results_code1 = pickle.load(open('/mbpp_results2/%d_code.pickle' % i, 'rb'))
    
    for seed in range(1):
        for k in range(5):
            he_prompt = remove_few_shot(problems['MBPP/%d' % i]['prompt'])
            he_generation = results_code1[('MBPP/%d' % i, seed)][0][k]
            
            prompt = SELF_EVAL_PROMPT.replace('[PROBLEM]', he_prompt)
            prompt = prompt.replace('[ANSWER]', he_generation)
            tasks.append(prompt)

In [256]:
pickle.dump(tasks, open('self_eval_mbpp_tasks.pickle', 'wb'))

In [257]:
len(tasks)

2550

In [258]:
A = pickle.load(open('/self_eval_mbpp/%d.pickle' % 0, 'rb'))

In [259]:
A

[' A. True\n\nProblem: You are an expert Python programmer, and here is your task: Write a function to find the minimum cost path to reach (']

In [260]:
res = []

for i in range(2550):
    answer = pickle.load(open('/self_eval_mbpp/%d.pickle' % i, 'rb'))

    res.append(answer[0].strip(' ').strip('\n')[0])

In [261]:
pickle.dump(res, open('/hcode/self_eval_mbpp_res.pickle', 'wb'))

### Create dataset for HE

In [268]:
SELF_EVAL_PROMPT = '''Instruction: Please evaluate the truthfulness of the proposed answer based on the given problem and internal knowledge.
Problem: [PROBLEM]
Proposed Answer: [ANSWER]
Is the proposed answer:
A. True
B. False
The proposed answer is:'''

In [269]:
problems = pickle.load(open('/he_problems.pickle', 'rb'))

In [270]:
NUM_TASKS = 164

In [271]:
tasks = []

for i in range(NUM_TASKS):
    results_code1 = pickle.load(open('/he_results2/%d_code.pickle' % i, 'rb'))
    
    for seed in range(5):
        for k in range(5):
            he_prompt = problems['HumanEval/%d' % i]['prompt']
            he_generation = results_code1[('HumanEval/%d' % i, seed)][0][k]
            
            prompt = SELF_EVAL_PROMPT.replace('[PROBLEM]', extract_triple_quoted_text(he_prompt))
            prompt = prompt.replace('[ANSWER]', extract_outside_triple_quotes_v2(he_prompt) + '\n' + he_generation)
            tasks.append(prompt)

In [272]:
pickle.dump(tasks, open('self_eval_he_tasks.pickle', 'wb'))

In [273]:
def extract_triple_quoted_text(text):
    """
    Extracts all text blocks enclosed in triple quotes (\"\"\" \"\"\") from the input text.
    
    Args:
        text (str): The input string to search for triple-quoted blocks.
    
    Returns:
        list: A list of all matched triple-quoted text blocks (without the quotes).
    """
    pattern = r'"""(.*?)"""'
    # Using re.DOTALL flag to make . match newlines as well
    matches = re.findall(pattern, text, re.DOTALL)
    return ''.join([match.strip() for match in matches])

In [274]:
def extract_outside_triple_quotes_v2(text):
    """
    Alternative implementation using split()
    """
    # Split on triple quotes and take every other segment
    segments = re.split(r'\"\"\"', text)
    # The segments outside quotes are at even indices (0, 2, 4...)
    outside_segments = [segments[i].strip() for i in range(0, len(segments), 2)]
    return ''.join([seg for seg in outside_segments if seg])  # Filter empty strings

In [275]:
res = []

for i in range(4100):
    answer = pickle.load(open('/self_eval_he/%d.pickle' % i, 'rb'))

    res.append(answer[0].strip(' ').strip('\n')[0])

In [276]:
pickle.dump(res, open('/hcode/self_eval_he_res.pickle', 'wb'))